# Jupyter Notebook Cells

# [1] Imports

In [ ]:
import torch
import pandas as pd
from model import PollutionGNN, GNNDataLoader
from torch_geometric.loader import DataLoader

# [2] Load Data

In [ ]:
df = pd.read_csv('../data_processing/wind_patterns.csv')
preprocessor = AirQualityPreprocessor()
train_df, test_df = preprocessor.train_test_split(df)

# [3] Create Graph Dataset

In [ ]:
loader = GNNDataLoader(sensor_locs)
train_data = [loader.create_graph(row) for row in train_df.iterrows()]
test_data = [loader.create_graph(row) for row in test_df.iterrows()]

# [4] Initialize Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PollutionGNN().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
loss_fn = nn.HuberLoss()

# [5] Training Loop

In [ ]:
for epoch in range(100):
    model.train()
    for batch in DataLoader(train_data, batch_size=32):
        optimizer.zero_grad()
        out = model(batch.to(device))
        loss = loss_fn(out, batch.y)
        loss.backward()
        optimizer.step()

# Validation

In [ ]:
   model.eval()
    with torch.no_grad():
        val_loss = sum(loss_fn(model(batch), batch.y) 
                    for batch in test_data)
    print(f"Epoch {epoch}: Loss {loss.item():.4f} | Val Loss {val_loss:.4f}")

# [6] Save Model
torch.save(model.state_dict(), 'pollution_gnn.pth')